#### Importing the required libraries

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from math import hypot
import pyautogui
import dlib

## Step 1: Face Detection

### Initialize The DNN Module:

In [2]:
# Path to the weights file
model_weights =  "model/res10_300x300_ssd_iter_140000_fp16.caffemodel"

# Path to the architecture file
model_arch = "model/deploy.prototxt.txt"

# Load the caffe model
net = cv2.dnn.readNetFromCaffe(model_arch, model_weights)

### Create A Face Detection Function:

In [3]:
def face_detector(image, threshold =0.7):
    
    # Get the height,width of the image
    h, w = image.shape[:2]

    # Apply mean subtraction, and create 4D blob from image
    blob = cv2.dnn.blobFromImage(image, 1.0,(300, 300), (104.0, 117.0, 123.0))
    
    # Set the new input value for the network
    net.setInput(blob)
    
    # Run forward pass on the input to compute output
    faces = net.forward()
    
    # Get the confidence value for all detected faces
    prediction_scores = faces[:,:,:,2]
    
    # Get the index of the prediction with highest confidence 
    i = np.argmax(prediction_scores)
    
    # Get the face with highest confidence 
    face = faces[0,0,i]
    
    # Extract the confidence
    confidence = face[2]
    
    # if confidence value is greater than the threshold
    if confidence > threshold:
        
        # The 4 values at indexes 3 to 6 are the top-left, bottom-right coordinates
        # scales to range 0-1.The original coordinates can be found by 
        # multiplying x,y values with the width,height of the image
        box = face[3:7] * np.array([w, h, w, h])
        
        # The coordinates are the pixel numbers relative to the top left
        # corner of the image therefore needs be quantized to int type
        (x1, y1, x2, y2) = box.astype("int")
        
        # Draw a bounding box around the face.
        annotated_frame = cv2.rectangle(image.copy(), (x1, y1), (x2, y2), (0, 0, 255), 2)
        output = (annotated_frame, (x1, y1, x2, y2), True, confidence)
    
    # Return the original frame if no face is detected with high confidence.
    else:
        output = (image,(),False, 0)
    
    
    return output

##### Test the **`face_detector()`** function with a real-time camera feed


In [4]:
# Get the video feed from webcam
cap = cv2.VideoCapture(0)

# Set the window to a normal one so we can adjust it
cv2.namedWindow('face Detection', cv2.WINDOW_NORMAL) 

while(True):
    
    # Read the frames
    ret, frame = cap.read()
    
    # Break if frame is not returned
    if not ret:
        break
        
    # Flip the frame
    frame = cv2.flip( frame, 1 )
    
    # Detect face in the frame
    annotated_frame, coords, status, conf = face_detector(frame)
    
    # Display the frame
    cv2.imshow('face Detection', annotated_frame)
    
    # Break the loop if 'q' key pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture and destroy the window
cap.release()
cv2.destroyAllWindows()

## Step 2: Landmarks Detection

#### Set Up the facial landmark Detector:

Download [detection model from here](https://github.com/italojs/facial-landmarks-recognition/blob/master/shape_predictor_68_face_landmarks.dat) and put it inside the `model` folder inside this directory.

In [5]:
# initialize the landmark detector
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

### Create the detect_landmarks() function

In [6]:
def detect_landmarks(box, image):
    
    # For faster results convert the image to gray-scale
    gray_scale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Get the coordinates 
    (x1, y1, x2, y2) = box

    # Perform the detection
    shape = predictor(gray_scale, dlib.rectangle(x1, y1, x2, y2))
    
    # Get the numPy array containing the coordinates of the landmarks
    landmarks = shape_to_np(shape)
    
   # Draw the landmark points with circles 
    for (x, y) in landmarks:
        annotated_image = cv2.circle(image, (x, y),2, (0, 127, 255), -1)

    return annotated_image, landmarks

##### Create the **`shape_to_np()`** helper function. 

In [7]:
def shape_to_np(shape): 
    
    # Create an array of shape (68, 2) for storing the landmark coordinates
    landmarks = np.zeros((68, 2), dtype="int")
    
    # Write the x,y coordinates of each landmark into the array
    for i in range(0, 68): 
        landmarks[i] = (shape.part(i).x, shape.part(i).y)
        
        
    return landmarks

##### Test **`detect_landmarks()`** function:

In [8]:
# Get the video feed from webcam
cap = cv2.VideoCapture(0)

# Set the window to a normal one so we can adjust it
cv2.namedWindow('Landmark Detection', cv2.WINDOW_NORMAL) 

while(True):
    # Read the frames
    ret, frame = cap.read()
    
    # Break if frame is not returned
    if not ret:
        break
        
    # Flip the frame
    frame = cv2.flip( frame, 1 )
    
    # Detect face in the frame
    face_image, box_coords, status, conf = face_detector(frame)
    
    if status:
        
        # Get the landmarks for the face region in the frame
        landmark_image, landmarks = detect_landmarks(box_coords, frame)

    # Display the frame
    cv2.imshow('Landmark Detection',landmark_image)
    
    # Break the loop if 'q' key pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture and destroy the window
cap.release()
cv2.destroyAllWindows()

## Step 3: Jump Control mechanism

In [9]:
def is_mouth_open(landmarks, ar_threshold = 0.7): 
    
    
    # Calculate the euclidean distance labelled as A,B,C
    A = hypot(landmarks[50][0] - landmarks[58][0], landmarks[50][1] - landmarks[58][1])
    B = hypot(landmarks[52][0] - landmarks[56][0], landmarks[52][1] - landmarks[56][1])
    C = hypot(landmarks[48][0] - landmarks[54][0], landmarks[48][1] - landmarks[54][1])
    
    # Calculate the mouth aspect ratio
    # The value of vertical distance A,B is averaged
    mouth_aspect_ratio = (A + B) / (2.0 * C)
    
    # Return True if the value is greater than the threshold
    if mouth_aspect_ratio > ar_threshold:
        return True, mouth_aspect_ratio
    else:
        return False, mouth_aspect_ratio

##### Testing the **`is_mouth_open()`** function:

In [10]:
# Get the video feed from webcam
cap = cv2.VideoCapture(0)

# Set the window to a normal one so we can adjust it
cv2.namedWindow('Mouth Status', cv2.WINDOW_NORMAL)

while(True):
    # Read the frames
    ret, frame = cap.read()
    
    # Break if frame is not returned
    if not ret:
        break
        
    # Flip the frame
    frame = cv2.flip( frame, 1 )
    
    # Detect face in the frame
    face_image, box_coords, status, conf = face_detector(frame)
    
    if status:
        
        # Get the landmarks for the face region in the frame
        landmark_image, landmarks = detect_landmarks(box_coords, frame)
        
        # Adjust the threshold and make sure it's working for you.
        mouth_status,_ = is_mouth_open(landmarks, ar_threshold = 0.6)
        
        # Display the mouth status
        cv2.putText(frame,'Is Mouth Open: {}'.format(mouth_status),
                    (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 127, 255),2)


    # Display the frame
    cv2.imshow('Mouth Status',frame)
    
    # Break the loop if 'q' key pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture and destroy the window
cap.release()
cv2.destroyAllWindows()

## Step 4: Crouch Control Mechanism

In [11]:
def face_proximity(box,image, proximity_threshold = 325):
    
    # Get the height and width of the face bounding box
    face_width =  box[2]-box[0]
    face_height = box[3]-box[1]
    
    # Draw rectangle to guide the user 
    # Calculate the angle of diagonal using face width, height 
    theta = np.arctan(face_height/face_width)
     
    # Use the angle to calculate height, width of the guide rectangle
    guide_height = np.sin(theta)*proximity_threshold
    guide_width  = np.cos(theta)*proximity_threshold
    
    # Calculate the mid-point of the guide rectangle/face bounding box
    mid_x,mid_y = (box[2]+box[0])/2 , (box[3]+box[1])/2
    
    #Calculate to coordinates of top-left and bottom-right corners
    guide_topleft = int(mid_x-(guide_width/2)), int(mid_y-(guide_height/2))
    guide_bottomright = int(mid_x +(guide_width/2)), int(mid_y + (guide_height/2))
    
    # Draw the guide rectangle
    cv2.rectangle(image, guide_topleft, guide_bottomright, (0, 255, 255), 2)
    
    # Calculate the diagonal distance of the bounding box
    diagonal = hypot(face_width, face_height)
    
    # Return True if distance greater than the threshold
    if diagonal > proximity_threshold:
        return True, diagonal
    else:
        return False, diagonal


##### Testing the **`face_proximity()`** function:

In [12]:
# Get the video feed from webcam
cap = cv2.VideoCapture(0)

# Set the window to a normal one so we can adjust it
cv2.namedWindow('Face proximity', cv2.WINDOW_NORMAL)

while(True):
    
    # Read the frames
    ret, frame = cap.read()
    
    # Break if frame is not returned
    if not ret:
        break
        
    # Flip the frame
    frame = cv2.flip( frame, 1 )
    
    # Detect face in the frame
    face_image, box_coords, status, conf = face_detector(frame)
    
    if status:
        
        # Check if face is closer than the defined threshold
        is_face_close,_ = face_proximity(box_coords, face_image, proximity_threshold = 325)
        
        # Display the mouth status
        cv2.putText(face_image,'Is Face Close: {}'.format(is_face_close),
                    (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 127, 255),2)

        
    # Display the frame
    cv2.imshow('Face proximity',face_image)
    
    # Break the loop if 'q' key pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture and destroy the window
cap.release()
cv2.destroyAllWindows()

## Step 5: Perform Calibration (Optional)

In [13]:
# Get the video feed from webcam
cap = cv2.VideoCapture(0)

# Set the window to a normal one so we can adjust it
cv2.namedWindow('Calibration', cv2.WINDOW_NORMAL)

while(True):
    
    # Read the frames
    ret, frame = cap.read()
    
    # Break if frame is not returned
    if not ret:
        break
        
    # Flip the frame
    frame = cv2.flip( frame, 1 )
    
    # Detect face in the frame
    face_image, box_coords, status, conf = face_detector(frame)
    
    if status:
        
        # Detect landmarks if the frame is found
        landmark_image, landmarks = detect_landmarks(box_coords, frame)
        
        # Get the current mouth aspect ratio
        _,mouth_ar = is_mouth_open(landmarks)
    
        # Get the current face proximity
        _, proximity  = face_proximity(box_coords, face_image)

        # Calculate threshold values
        ar_threshold = mouth_ar*1.4
        proximity_threshold = proximity*1.3

        
        # Dsiplay the threshold values
        cv2.putText(frame, 'Aspect ratio threshold: {:.2f} '.format(ar_threshold), 
                    (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 127, 255),2)
        
        cv2.putText(frame,'Proximity threshold: {:.2f}'.format(proximity_threshold), 
                    (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 127, 255),2)
     
    # Display the frame
    cv2.imshow('Calibration',frame)
    
    # Break the loop if 'q' key pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture and destroy the window
cap.release()
cv2.destroyAllWindows()

## Step 6: Keyboard Automation
***Note:*** *When running the following cells in Jupyter Notebook, make sure you don't use the **Shift + Enter** command to run the following code cells. You can use the Run code cell button in the Toolbar.* *Also if the keyboard buttons misbehave then you can also restart the kernel*

In [14]:
# This will open a context menu
pyautogui.click(button='right')

In [15]:
# Press space bar. This will scroll down the page in some browsers
pyautogui.press('space')

To press multiple keys we can pass a list of strings to **`press()`** function.

In [16]:
# This will move the focus to the next cell in the notebook
pyautogui.press(['shift','enter'])

When you use **`pyautogui.keyDown()`**  instead of **`pyautogui.press()`**. Then the specified key is held down unless  **`pyautogui.keyUp()`**event takes place helping us trigger a longer key press.

In [17]:
# Hold down the shift key
pyautogui.keyDown('shift')

In [19]:
# Press enter while the shift key is down, this will run the next code cell
pyautogui.press('enter')



# Release the shift key
pyautogui.keyUp('shift')

In [14]:

# This will run automatically after running the two code cells above
print('I ran')

I ran


## Step 7:  Build The Final Application

Go to **`Chrome://Dino`** in your Chrome browser and run the code cell below.

***Note:*** *The image window screen will freeze when you trigger key buttons since at that moment the while loop will pause to press the key. So don't worry about that, after the program launches minimize the camera window and just go to chrome://dino/ and start playing using your face and mouth.*

In [14]:
# 添加頭部左右轉動檢測功能
def detect_head_turn(landmarks, threshold=0.05):
    """
    檢測頭部左右轉動
    使用鼻尖相對於臉部中心的位置來判斷頭部轉向
    
    參數:
    landmarks - 面部68個關鍵點
    threshold - 偏移閾值，超過此值判定為頭部轉向
    
    返回:
    direction - 轉向方向 ("left", "right", "center")
    ratio - 偏移比例，用於判斷轉向程度
    """
    # 計算臉部中心位置（兩側臉頰中點）
    face_left = landmarks[0][0]  # 左臉頰點
    face_right = landmarks[16][0]  # 右臉頰點
    face_center_x = (face_left + face_right) / 2
    
    # 獲取鼻尖位置
    nose_tip_x = landmarks[30][0]
    
    # 計算鼻尖相對於臉部中心的偏移比例
    # 標準化為臉部寬度的比例
    face_width = abs(face_right - face_left)
    shift_ratio = (nose_tip_x - face_center_x) / face_width
    
    # 判斷頭部轉向
    if shift_ratio < -threshold:
        return "left", shift_ratio
    elif shift_ratio > threshold:
        return "right", shift_ratio
    else:
        return "center", shift_ratio

# 在主循環中整合頭部轉向檢測
# 替換原本的主循環
# Get the video feed from webcam
cap = cv2.VideoCapture(0)

# Set the window to a normal one so we can adjust it
cv2.namedWindow('Dino with OpenCV', cv2.WINDOW_NORMAL)

# By default each key press is followed by a 0.1 second pause
pyautogui.PAUSE = 0.0

# 設定頭部轉動的閾值
head_turn_threshold = 0.07  # 可以根據實際情況調整

# 設定校準階段
print("保持頭部正向，進行校準...")
calibration_frames = 30
head_positions = []

# 校準階段收集正常頭部位置數據
for _ in range(calibration_frames):
    ret, frame = cap.read()
    if not ret:
        break
        
    frame = cv2.flip(frame, 1)
    face_image, box_coords, status, conf = face_detector(frame)
    
    if status:
        landmark_image, landmarks = detect_landmarks(box_coords, frame)
        _, shift_ratio = detect_head_turn(landmarks, 0)
        head_positions.append(shift_ratio)
        
        cv2.putText(frame, f"校準中: {len(head_positions)}/{calibration_frames}", 
                   (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 255, 0), 2)
        
    cv2.imshow('Dino with OpenCV', frame)
    cv2.waitKey(1)

# 計算個人化頭部轉動閾值
if head_positions:
    avg_position = sum(head_positions) / len(head_positions)
    head_turn_threshold = 2.0 * max(0.03, abs(avg_position))  # 最小閾值為0.03
    print(f"校準完成，頭部轉動閾值: {head_turn_threshold}")

print("開始遊戲控制！使用:")
print("- 張嘴跳躍")
print("- 靠近攝像頭下蹲")
print("- 頭部左轉向左移動")
print("- 頭部右轉向右移動")

while(True):
    # Read the frames
    ret, frame = cap.read()
    
    # Break if frame is not returned
    if not ret:
        break
        
    # Flip the frame
    frame = cv2.flip(frame, 1)
    
    # Detect face in the frame
    face_image, box_coords, status, conf = face_detector(frame)
    
    if status:
        # Detect landmarks if a face is found
        landmark_image, landmarks = detect_landmarks(box_coords, frame)
        
        # 1. 檢測嘴巴狀態控制跳躍 (空格鍵)
        is_open, _ = is_mouth_open(landmarks, ar_threshold)
        if is_open:
            pyautogui.keyDown('up')
            mouth_status = 'Open'
        else:
            pyautogui.keyUp('up')
            mouth_status = 'Closed'
        
        # 2. 檢測臉部靠近程度控制下蹲 (下鍵)
        is_closer, _ = face_proximity(box_coords, frame, proximity_threshold)
        if is_closer:
            pyautogui.keyDown('down')
            proximity_status = 'Close'
        else:
            pyautogui.keyUp('down')
            proximity_status = 'Far'
        
        # 3. 新增：檢測頭部左右轉動控制左右移動 (左右鍵)
        head_direction, turn_ratio = detect_head_turn(landmarks, head_turn_threshold)
        
        # 根據頭部轉向控制鍵盤
        if head_direction == "left":
            # 按下左鍵或A鍵 (可選擇其一)
            pyautogui.keyDown('left')  # 或使用 'a'
            pyautogui.keyUp('right')   # 確保右鍵被釋放
            turn_status = 'Left'
        elif head_direction == "right":
            # 按下右鍵或D鍵 (可選擇其一)
            pyautogui.keyDown('right')  # 或使用 'd'
            pyautogui.keyUp('left')     # 確保左鍵被釋放
            turn_status = 'Right'
        else:
            # 如果頭部居中，釋放左右鍵
            pyautogui.keyUp('left')
            pyautogui.keyUp('right')
            turn_status = 'Center'
        
        # 在畫面上顯示所有控制狀態
        cv2.putText(frame, f'Mouth: {mouth_status}', 
                   (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 127, 255), 2)
        cv2.putText(frame, f'Face: {proximity_status}', 
                   (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 127, 255), 2)
        cv2.putText(frame, f'Head: {turn_status} ({turn_ratio:.2f})', 
                   (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 127, 255), 2)
        
        # 畫出頭部轉向輔助線
        # 臉部中心線
        face_center_x = int((landmarks[0][0] + landmarks[16][0]) / 2)
        cv2.line(frame, (face_center_x, landmarks[27][1]), 
                (face_center_x, landmarks[8][1]), (0, 255, 0), 1)
        
        # 鼻子方向線
        nose_line_length = 30
        nose_vector_x = landmarks[30][0] - landmarks[27][0]
        nose_vector_y = landmarks[30][1] - landmarks[27][1]
        nose_length = max(1, hypot(nose_vector_x, nose_vector_y))
        nose_direction_x = int(landmarks[30][0] + nose_vector_x/nose_length * nose_line_length)
        nose_direction_y = int(landmarks[30][1] + nose_vector_y/nose_length * nose_line_length)
        cv2.line(frame, (landmarks[30][0], landmarks[30][1]), 
                (nose_direction_x, nose_direction_y), (0, 0, 255), 2)
    
    # Display the frame
    cv2.imshow('Dino with OpenCV', frame)
    
    # Break the loop if 'q' key pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

保持頭部正向，進行校準...
校準完成，頭部轉動閾值: 0.2708498152066725
開始遊戲控制！使用:
- 張嘴跳躍
- 靠近攝像頭下蹲
- 頭部左轉向左移動
- 頭部右轉向右移動
